# ChatGLM3 Lora 实战

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

## Step1 导入相关包

In [2]:
from datasets import Dataset
from transformers import AutoModelForCausalLM # chatglm3就是自回归模型，用因果语言模型加载也不奇怪
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chatglm3-6b-base/", trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='/data/PLM/chatglm3-6b-base/', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	
}

In [6]:
# 老样子，直接对eos_token进行tokenize会把它再次分词，而且会加上[gMASK]sop的前缀
print(tokenizer(tokenizer.eos_token))
print(tokenizer.eos_token_id)

{'input_ids': [64790, 64792, 2893, 30917, 30994], 'attention_mask': [1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4]}
2


In [7]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = "\n".join([example["instruction"], example["input"]]).strip()     # query
    # tokenizer.build_chat_input()自带[gMASK]sop的前缀，decode时也会显示出来
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n query<|assistant|>
    # [EOS]在chatglm分词器中也不会自带，即使add_special_tokens=True，必须手动添加，且decode时不会显示出来，即使skip_special_tokens=False
    response = tokenizer("\n" + example["output"], add_special_tokens=False)        # \n response, 缺少eos token。这里必须把add_special_tokens设置成False，不然会把[gMASK]sop等前缀也加进来！
    # 注意tokenizer.build_chat_input()返回的是tensor而非list，所以需要转换成list再相加！
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [9]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK]sop<|user|> \n 解释为什么以下分数等同于1/4\n输入：4/16<|assistant|> \n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

In [10]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'\n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

## Step4 创建模型

In [11]:
import torch
# 仅使用torch_dtype=torch.half并不经过量化和反量化过程！只是改变了权重的范围！
model = AutoModelForCausalLM.from_pretrained("/data/PLM/chatglm3-6b-base/", trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.half, device_map="auto")

Loading checkpoint shards: 100%|██████████| 7/7 [00:14<00:00,  2.11s/it]


In [12]:
for name, param in model.named_parameters():
    print(name)

transformer.embedding.word_embeddings.weight
transformer.encoder.layers.0.input_layernorm.weight
transformer.encoder.layers.0.self_attention.query_key_value.weight
transformer.encoder.layers.0.self_attention.query_key_value.bias
transformer.encoder.layers.0.self_attention.dense.weight
transformer.encoder.layers.0.post_attention_layernorm.weight
transformer.encoder.layers.0.mlp.dense_h_to_4h.weight
transformer.encoder.layers.0.mlp.dense_4h_to_h.weight
transformer.encoder.layers.1.input_layernorm.weight
transformer.encoder.layers.1.self_attention.query_key_value.weight
transformer.encoder.layers.1.self_attention.query_key_value.bias
transformer.encoder.layers.1.self_attention.dense.weight
transformer.encoder.layers.1.post_attention_layernorm.weight
transformer.encoder.layers.1.mlp.dense_h_to_4h.weight
transformer.encoder.layers.1.mlp.dense_4h_to_h.weight
transformer.encoder.layers.2.input_layernorm.weight
transformer.encoder.layers.2.self_attention.query_key_value.weight
transformer.enco

## Lora

### PEFT Step1 配置文件

In [13]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, PeftModelForCausalLM

config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["query_key_value"]) # chatglm不是peft中预置的模型，所以需要指定target_modules，即在哪些权重上加Lora
config # task_type必须指定，不然模型训练会报错，其原因在于指定之后模型类型就会从PeftModel转化为PeftModelForCausalLM，将forward的参数显式地指定出来，不然TrainingArguments中的remove_unused_columns只会把它们全部清除导致Invalid key: 5122 is out of bounds for size 0这种鬼错误

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

### PEFT Step2 创建模型

In [14]:
model = get_peft_model(model, config)

In [15]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/data/PLM/chatglm3-6b-base/', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [16]:
for name, parameter in model.named_parameters():
    print(name) # 注意这里没有q, k, v之分，只有query_key_value，所以只能仅对它用Lora。同时这里是有weight和bias之分的

base_model.model.transformer.embedding.word_embeddings.weight
base_model.model.transformer.encoder.layers.0.input_layernorm.weight
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.weight
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.bias
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_A.default.weight
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_B.default.weight
base_model.model.transformer.encoder.layers.0.self_attention.dense.weight
base_model.model.transformer.encoder.layers.0.post_attention_layernorm.weight
base_model.model.transformer.encoder.layers.0.mlp.dense_h_to_4h.weight
base_model.model.transformer.encoder.layers.0.mlp.dense_4h_to_h.weight
base_model.model.transformer.encoder.layers.1.input_layernorm.weight
base_model.model.transformer.encoder.layers.1.self_attention.query_key_value.weight
base_model.model.transformer.encoder.layers.1.self_attenti

In [17]:
model.print_trainable_parameters()

trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614


In [18]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): Linear(
                  in_features=4096, out_features=4608, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False)
                  )
                  (lora_embedding_A)

## Step5 配置训练参数

In [19]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4,
    remove_unused_columns=False # 默认为True，设置为False后config = LoraConfig()就可以不设task_type参数了
)

## Step6 创建训练器

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [21]:
trainer.train()

Step,Training Loss
10,1.925400
20,1.847500
30,1.856600
40,1.735500
50,1.641300
60,1.586800
70,1.590700
80,1.587800
90,1.604600
100,1.608800


TrainOutput(global_step=187, training_loss=1.6415953814664626, metrics={'train_runtime': 424.1871, 'train_samples_per_second': 14.145, 'train_steps_per_second': 0.441, 'total_flos': 2.1587884962816e+16, 'train_loss': 1.6415953814664626, 'epoch': 1.0})

## Step8 模型推理

In [22]:
model.eval()
print(model.chat(tokenizer, "数学考试怎么考高分？", history=[])[0])

数学考试要取得高分，需要以下几个步骤：
1. 认真复习：首先，要仔细复习数学课程，了解所有概念和公式，确保理解透彻。
2. 练习：做大量的练习题，包括课本习题、模拟试题和历年真题，提高自己的解题能力。
3. 注意细节：在考试中，注意细节，仔细审题，避免粗心大意，减少失分机会。
4. 掌握解题技巧：学习一些解题技巧，如代入法、图像法、排除法等，可以提高解题效率。
5. 考前放松：考试前适当放松，保持良好的心态，减少紧张情绪，有助于提高发挥水平。

希望以上建议能对您有所帮助，祝您在数学考试中取得好成绩！
